In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

spark = SparkSession \
    .builder \
    .appName('Stream Demo') \
    .getOrCreate()

spark 



In [ ]:
from pyspark.sql.functions import *

weather_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "weather-data") \
    .option("startingOffsets", "latest") \
    .load()
    

traffic_stream = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "traffic-data") 
    .load()

weather_json_df = weather_stream.withColumn('value', expr('cast(value as string)')).withColumn('key', expr('cast(key as string)'))

weather_schema = StructType([
    StructField('Longitude', DoubleType()),
    StructField('Latitude', DoubleType()),
    StructField('weather', StringType()),
    StructField('weather_description', StringType()),
    StructField('temp', DoubleType()),
    StructField('visibility', IntegerType()),
    StructField('clouds', IntegerType()),
    StructField('rain', IntegerType()),
    StructField('snow', IntegerType()),
    StructField('date', IntegerType()),
    StructField('name', StringType())
])

weather_flatten_df = df.select("key",
                df['values_json.Longitude'].alias('longitude'), 
                df['values_json.Latitude'].alias('latitude'), 
                df['values_json.weather'].alias('weather'), 
                df['values_json.weather_description'].alias('weather_description'), 
                df['values_json.temp'].alias('temp'), 
                df['values_json.visibility'].alias('visibility'),
                df['values_json.clouds'].alias('clouds'),
                df['values_json.rain'].alias('rain'),
                df['values_json.snow'].alias('snow'),
                df['values_json.date'].alias('date'),
                df['values_json.name'].alias('name'))

#weather_flatten_df.show()
#flatten_df.printSchema()

In [17]:
query = flatten_df.writeStream \
    .outputMode('update') \
    .format('console') \
    .start()

query.awaitTermination()


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
traffic_json_df = traffic_stream.withColumn('value', expr('cast(value as string)')).withColumn('key', expr('cast(key as string)'))

traffic_json_df.show()